# 🩺 Pneumonia Detection using InceptionResNetV2
This notebook trains a deep learning model using **InceptionResNetV2** to detect Pneumonia from chest X-ray images, and deploys it with **Gradio**.

In [ ]:

# Import libraries
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
import os
import matplotlib.pyplot as plt


## 📂 Data Preparation

In [ ]:

# Data generators
data_generator = ImageDataGenerator(
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

# Directory paths
TRAINING_DIR = '/kaggle/input/pneumonia-polenzo/dataset/chest_xray/train'
TEST_DIR = '/kaggle/input/pneumonia-polenzo/dataset/chest_xray/test'
im_shape = (299, 299)
seed = 10
BATCH_SIZE = 32

# Create generators
train_generator = data_generator.flow_from_directory(
    TRAINING_DIR, 
    target_size=im_shape, 
    shuffle=True, 
    seed=seed,
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    subset="training"
)

validation_generator = val_data_generator.flow_from_directory(
    TRAINING_DIR, 
    target_size=im_shape, 
    shuffle=False, 
    seed=seed,
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    subset="validation"
)

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_generator.flow_from_directory(
    TEST_DIR, 
    target_size=im_shape, 
    shuffle=False, 
    seed=seed,
    class_mode='categorical', 
    batch_size=BATCH_SIZE
)

# Get dataset info
nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
nb_test_samples = test_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: ' + str(classes))
num_classes = len(classes)


## 🧠 Model Creation

In [ ]:

# Model creation
base_model = InceptionResNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(im_shape[0], im_shape[1], 3)
)

x = base_model.output
x = Flatten()(x)
x = Dense(200, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freezing pretrained layers
for layer in base_model.layers:
    layer.trainable = False
    
# Compile model
optimizer = Adam()
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


## 🏋️ Training the Model

In [ ]:

# Training setup
epochs = 10

# Callbacks
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.keras',
        monitor='val_loss', 
        save_best_only=True, 
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=10,
        verbose=1
    )
]

# Training
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // BATCH_SIZE,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator,
    verbose=1,
    validation_steps=nb_validation_samples // BATCH_SIZE
)


## 📊 Training History Visualization

In [ ]:

# Plot training history
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs_x = range(1, len(loss_values) + 1)
plt.figure(figsize=(6, 6))
plt.plot(epochs_x, loss_values, 'r', label='Training loss')
plt.plot(epochs_x, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('EpochVsLoss.jpg', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(6, 6))
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
plt.plot(epochs_x, acc_values, 'r', label='Training acc')
plt.plot(epochs_x, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('EpochVsAcc.jpg', dpi=300, bbox_inches='tight')
plt.show()


## 🚀 Gradio Deployment

In [ ]:

# Install and import Gradio
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2

# Load model
model = tf.keras.models.load_model('model.keras')  

# Class names
class_names = ["Normal", "Pneumonia"]  

# Preprocessing function
def preprocess_image(image):
    img = cv2.resize(image, (299, 299))  
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Prediction function
def predict(image):
    img_array = preprocess_image(image)
    preds = model.predict(img_array)

    if preds.shape[1] == 1:
        confidence = float(preds[0][0])
        label = class_names[1] if confidence > 0.5 else class_names[0]
        confidence_display = confidence if confidence > 0.5 else 1 - confidence
    else:
        class_index = int(np.argmax(preds[0]))
        label = class_names[class_index]
        confidence_display = float(np.max(preds[0]))

    return label, f"Confidence: {confidence_display:.2f}"

# Create Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy"),
    outputs=["text", "text"],
    title="🩺 Pneumonia Detection",
    description="Upload a chest X-ray image. The model will predict whether the image shows signs of Pneumonia or is Normal."
)

# Launch interface
iface.launch()
